In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [19]:
STORM_NUMBER = 2
TITLE = "28 December 2021"
SUBTITLE = "8 AM 29 December 2021"
sts = utc(2021, 12, 28, 0)
ets = utc(2021, 12, 29, 20, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 50, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [14]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [22]:
cull = [651, 'IA-BT-11', 'IA-BT-16', 'GLDI2']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [

]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=3)
res = mp.postprocess(filename='211229.png')
mp.close()

     state  wfo   val       lon      lat                              geo  \
0       WY  CYS  42.0 -106.7100  41.0600  POINT (-562150.128 -415119.858)   
1       WY  CYS  38.0 -106.3500  41.3800  POINT (-529418.416 -382020.879)   
2       WY  CYS  32.0 -106.2300  41.3600  POINT (-519600.362 -385004.234)   
3       WY  CYS  32.0 -106.9200  41.0100  POINT (-580142.560 -419207.250)   
4       WY  CYS  28.0 -106.2300  41.2400  POINT (-520575.681 -398314.232)   
...    ...  ...   ...       ...      ...                              ...   
3221    IL  ILX   0.0  -87.9747  39.2972  POINT (1031585.392 -558946.480)   
3222    IL  ILX   0.0  -89.4162  39.8563   POINT (901288.867 -514057.954)   
3224    IL  DVN   0.0  -89.7100  41.7800   POINT (850799.311 -304594.247)   
3225    IL  DVN   0.0  -89.4833  41.3833   POINT (874877.249 -346056.344)   
3226    IL  DVN   0.0  -91.0167  41.4167   POINT (747608.492 -357502.409)   

      used_for_analysis  nwsli  plotme source  
0                  True    